<a href="https://colab.research.google.com/github/joemunozb/Taller-Ing-Ambiental-y-Ciencia-de-Datos/blob/main/Taller_Ing_Amb_y_Ciencia_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **TALLER PRÁCTICO:**
## **Introducción al análisis de datos Hidrológicos con Python**
\
**Jorge Enrique Muñoz Barragán** \
jmunozb@ucentral.edu.co \
Docente \
Facultad de Ingeniería y Ciencias Básicas \
**Universidad Central**


[Pregrado en Ingeniería Ambiental - Universidad Central](https://www.ucentral.edu.co/programa-academico/ingenieria-ambiental)\
direccionambiental@ucentral.edu.co \
[Pregrado en Ciencia de Datos - Universidad Central](https://www.ucentral.edu.co/programa-academico/ciencia-datos) \
cienciadedatos@ucentral.edu.co \

## **Primera Parte del Taller: CURVA DE DURACIÓN DE CAUDALES**

1.   Curva de Duración de Caudales
2.   Carga de datos desde Google Drive
3.   Construcción de la Curva de Duración de Caudales





### Instalación de Liberias

In [ ]:
# Instalación de librerías necesarias
!pip install pandas numpy matplotlib scipy

# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Carga del Archivo


In [ ]:
# @title
# Autenticación y acceso a Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Ruta del archivo en Google Drive
file_path = '/content/drive/My Drive/caudales_medios_diarios.csv'

# Carga del archivo CSV
caudales_medios = pd.read_csv(file_path)

### Previsualizar los registros

In [ ]:
# Visualización del archivo completo
print(caudales_medios)

# Visualización de los primeros registros
print(caudales_medios.head())

# Verificación de la estructura del DataFrame
print(caudales_medios.info())

# Visualización de los datos de interés
print(caudales_medios[['Fecha', 'Valor']])

# Ordenar los caudales de mayor a menor
caudales_descendente = caudales_medios.sort_values(by='Valor', ascending=False)

print(caudales_descendente[['Fecha', 'Valor']])


### Crear una nueva columna en el archivo

In [ ]:
# Generar una columna de porcentajes de excedencia

#print(len(caudales_descendente))

caudales_descendente['excedencia'] = np.arange(1, len(caudales_descendente) + 1) / len(caudales_descendente)

print(caudales_descendente['excedencia'])



### Construcción de Curva de Duración de Caudales

In [ ]:
# Construcción de la curva de duración de caudales
plt.figure(figsize=(10, 6))
plt.plot(caudales_descendente['excedencia'], caudales_descendente['Valor'], marker='o', linestyle='-')
plt.xlabel('Porcentaje de Excedencia')
plt.ylabel('Caudal (m³/s)')
plt.title('Curva de Duración de Caudales')
plt.grid(True)
plt.show()

### Cálculo del "Q95"

En Ingeniería Ambiental, es muy usual utilizar el denominado Q95, o caudal de excedencia del 95% del tiempo. Representa un caudal que se presenta con un valor igual o mayor, o durante determinado tiempo (una probabiidad del 95% del tiempo), es decir, muy probable. Se utiliza para evaluar condiciones de calidad del agua, o límites de utilización del agua en un río.

In [ ]:
# Cálculo de Q95
Q95 = np.percentile(caudales_descendente['Valor'], 5) ##Se toma 5, porque los datos están en descendente.
print('El caudal Q95 (excedido el 95% del tiempo) es:', Q95, 'm³/s')

# Crear la figura y los ejes
plt.figure(figsize=(10, 6))

plt.plot(caudales_descendente['excedencia'], caudales_descendente['Valor'], marker='o', linestyle='-', label='Curva de Duración')

# Ajustar límites para que empiecen en cero
plt.xlim(left=0)
plt.ylim(bottom=0)

# Recalcular los límites del gráfico después de ajustar
xlim = plt.gca().get_xlim()
ylim = plt.gca().get_ylim()


plt.plot([0.95, 0.95], [ylim[0], Q95], color='r', linestyle='--', label=f'Q95 ({Q95:.2f} m³/s)')
plt.plot([xlim[0], 0.95], [Q95, Q95], color='b', linestyle='--', label=f'Caudal Q95 ({Q95:.2f} m³/s)')

plt.xlabel('Porcentaje de Excedencia')
plt.ylabel('Caudal (m³/s)')
plt.title('Curva de Duración de Caudales y Q95')
plt.legend()
plt.grid(True)

plt.show()


## **Segunda Parte del Taller: PRONÓSTICO DE EVENTOS EXTERMOS**

1.   Carga de datos
2.   Carga de datos desde Google Drive
3.   Construcción de la Curva de Duración de Caudales

In [ ]:
# Carga de Liberías

import numpy as np
import scipy.stats as stats
import pandas as pd


### Ejercicio:

Q_a = 7500  # m³/s
P_r = 60    # años

### Carga de Datos

# Registro de caudales

datos = {
    'Año': [1954, 1955, 1956, 1957, 1958 ,1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978],
    'Caudal': [2230, 3220, 2246, 1804, 2737, 2070, 3682, 4240, 2367, 7061, 2489, 2350, 3706, 2675, 6267, 5971, 4744, 6000, 4060, 6900, 5565, 3130, 2414, 1796, 7430]
}

## Número de años analizados

n = len(datos['Año'])

print('Número de años analizados', n, 'años')

# Crear un DataFrame de pandas o paquete de datos
caudales = pd.DataFrame(datos)

# Mostrar los caudales
print(caudales)

In [ ]:
## Ordenar datos en un nuevo data frame

# Creación de un nuevo dataframe de pandas:

  # Ordenar los datos de Caudal de mayor a menor

Caudal_desc = caudales['Caudal'].sort_values(ascending=False).reset_index(drop=True)

  # Crear una numeración
numeracion = pd.Series(range(1, len(Caudal_desc) + 1))

  # Crear el nuevo DataFrame

serie = pd.DataFrame({
    'Numer': numeracion,
    'Caudal': Caudal_desc
})

# Verificar el nuevo DataFrame
print(serie)

In [ ]:
## Aplicación de Funciones de Distribución de Probabilidad

 # FUNCIÓN DE DISTRIBUCIÓN DE PROBABILIDAD NORMAL

print('FUNCIÓN NORMAL:')

print('   - PARÁMETROS:')

       # Parámetro mu (Media aritmética)

mu_norm = serie['Caudal'].mean()
print("mu = ", mu_norm, 'm³/s')

       # Parámetro sigma (Desviación Estándar)

sigma_norm = serie['Caudal'].std()
print("sigma = ", sigma_norm, 'm³/s')

       # Variable estandarizada, z, para un caudal de 7500 m³/s

x = Q_a

z_nomr = (x - mu_norm) / sigma_norm
print("z = ", z_nomr)

print('     A) PROBABILIDAD DE QUE EN UN AÑO CUALQUIERA EL CAUDAL SEA MAYOR O IGUAL A', Q_a, 'm³/s')

      # Probabilidad de ocurrencia (Probabilidad de que el caudal máximo sea mayor o igual a 7500)

p_norm = stats.norm.cdf(z_nomr)

print('        La probabilidad de ocurrencia de un caudal igual o mayor a ', Q_a, 'm³/s es : p = ', p_norm)

      # Probabilidad de ocurrencia en años (Probabilidad de que el caudal máximo sea mayor o igual a 7500)

T_norm = 1 / (1-p_norm)

print('        Es probable que cada', T, 'años se registre un caudal igual o superior a ', Q_a, "m³/s (T = ", T_norm, "años)")

print('     B) EL CAUDAL DE DISEÑO PARA UN PERIODO DE RETORNO DE ', P_r, 'AÑOS:')

p_d = (P_r-1) / P_r

z_r = stats.norm.ppf(p_d)

print('        z = ', z_r)

x_r_norm = mu_norm + sigma_norm * z_r

print('        La probabiliad de un periodo de retorno de diseño de ', P_r, 'años es :', p_d)

print('        Por lo tanto, el caudal de diseño para un periodo de retorno de ', P_r, 'años es :', x_r_norm, ' años')



In [ ]:
import math

 # FUNCIÓN DE DISTRIBUCIÓN DE PROBABILIDAD LOG-NORMAL

print('FUNCIÓN LOG-NORMAL:')

print('   - PARÁMETROS:')

       # Serie_Logarítmica (Logaritmo de los datos)

serie_log= np.log(serie['Caudal'])
#print(serie_log)

       # Parámetro mu (Media aritmética)

alfa_log_norm = serie_log.mean()
print("alfa = ", alfa_log_norm)

       # Parámetro sigma (Desviación Estándar)

beta_log_norm = serie_log.std()
print("beta = ", beta_log_norm)

       # Variable estandarizada, z, para un caudal de 7500 m³/s

x_log = math.log(Q_a)
print("x_log = ", x_log)

z_log_nomr = (x_log - alfa_log_norm) / beta_log_norm
print("z = ", z_log_nomr)

print('     A) PROBABILIDAD DE QUE EN UN AÑO CUALQUIERA EL CAUDAL SEA MAYOR O IGUAL A', Q_a, 'm³/s')

      # Probabilidad de ocurrencia (Probabilidad de que el caudal máximo sea mayor o igual a 7500)

p_log_norm = stats.norm.cdf(z_log_nomr)

print('        La probabilidad de ocurrencia de un caudal igual o mayor a ', Q_a, 'm³/s es : p = ', p_log_norm)

      # Probabilidad de ocurrencia en años (Probabilidad de que el caudal máximo sea mayor o igual a 7500)

T_log_norm = 1 / (1-p_log_norm)

print('        Es probable que cada', T, 'años se registre un caudal igual o superior a ', Q_a, "m³/s (T = ", T_log_norm, "años)")

print('     B) EL CAUDAL DE DISEÑO PARA UN PERIODO DE RETORNO DE ', P_r, 'AÑOS:')

p_d = (P_r-1) / P_r

z_r_log = stats.norm.ppf(p_d)

print('        z = ', z_r_log)

x_r_log = math.exp(z_r_log * beta_log_norm + alfa_log_norm)

print('        La probabiliad de un periodo de retorno de diseño de ', P_r, 'años es :', p_d)

print('        Por lo tanto, el caudal de diseño para un periodo de retorno de ', P_r, 'años es :', x_r_log, ' años')



In [ ]:
import scipy.stats as st

 # FUNCIÓN DE DISTRIBUCIÓN DE PROBABILIDAD GUMBEL

print('FUNCIÓN GUMBEL:')

print('   - PARÁMETROS:')

       # Características de los datos

N = len(serie['Caudal'])
print('Número de datos', N)

Media = serie['Caudal'].mean()
print("Media = ", Media)

Desviacion = serie['Caudal'].std()
print("Desviación = ", Desviacion)

       # Ajuste a Función Gumbel

yi = [-np.log(np.log((N + 1) / (j + 1))) for j in range(N)]

       # Paráemtros de la función Gumbel

mu_gumbel = np.mean(yi)
sigma_gumbel = np.std(yi, ddof=0)
alfa_gumbel = sigma_gumbel/Desviacion
beta_gumbel = Media - (mu_gumbel/alfa_gumbel)

print("Parámetro de ubicación (mu):", mu_gumbel)
print("Parámetro de escala (sigma):", sigma_gumbel)
print("Parámetro alfa = ", alfa_gumbel)
print("Parámetro beta = ", beta_gumbel)

print('     A) PROBABILIDAD DE QUE EN UN AÑO CUALQUIERA EL CAUDAL SEA MAYOR O IGUAL A', Q_a, 'm³/s')

      # Probabilidad de ocurrencia (Probabilidad de que el caudal máximo sea mayor o igual a 7500)

p_gumbel = math.exp(-math.exp(-alfa_gumbel * (Q_a - beta_gumbel)))

print('        La probabilidad de ocurrencia de un caudal igual o mayor a ', Q_a, 'm³/s es : p = ', p_gumbel)

      # Probabilidad de ocurrencia en años (Probabilidad de que el caudal máximo sea mayor o igual a 7500)

T_gumbel = 1 / (1-p_log_norm)

print('        Es probable que cada', T, 'años se registre un caudal igual o superior a ', Q_a, "m³/s (T = ", T_gumbel, "años)")

print('     B) EL CAUDAL DE DISEÑO PARA UN PERIODO DE RETORNO DE ', P_r, 'AÑOS:')

p_d = (P_r-1) / P_r

x_r_gumbel = beta_gumbel - (1/alfa_gumbel) * math.log(math.log(P_r/(P_r-1)))

print('        La probabiliad de un periodo de retorno de diseño de ', P_r, 'años es :', p_d)

print('        Por lo tanto, el caudal de diseño para un periodo de retorno de ', P_r, 'años es :', x_r_gumbel, ' años')



In [ ]:
#Resumen de resultados:

resumen_resultados = {
    'Función': ['Normal', 'Log-Normal', 'Gumbel'],
    'Prob de Ocurrencia': [p_norm, p_log_norm, p_gumbel],
    'Prob en Años': [T_norm, T_log_norm, T_gumbel],
    'Caudal de Diseño': [x_r_norm, x_r_log, x_r_gumbel]
}

# Crear un DataFrame de los resultados

resultados = pd.DataFrame(resumen_resultados)

# Mostrar los resultados
print(resultados)

# PRUEBAS DE BONDAD DE AJUSTE

# Método del error cuadrático mínimo

In [ ]:
## Cálculo de caudales Ajustados a las FDP

# Creación de un nuevo dataframe de pandas

  # Número de años en la serie original

N = len(serie['Caudal'])
print('Número de años en la serie original:',N)

  # Cálculo de periodos de retorno

m = pd.Series(range(1, N + 1))
T = (N + 1) / m
teoric_p = 1 - (m / (N + 1))

# Crear un DataFrame que contenga todas las series
Caudales_ajustados = pd.DataFrame({
    'm': m,
    'T (años)': T,
    'P Teórica': teoric_p
})

print(Caudales_ajustados)

In [ ]:
  # Cálculo de Datos Ajustados a Función Normal
serie_z_norm = pd.Series(stats.norm.ppf(teoric_p))
norm_a = pd.Series(serie_z_norm*sigma_norm + mu_norm)
#print(norm_a)

  # Cálculo de Datos Ajustados a Función Normal
serie_z_log_norm = pd.Series(stats.norm.ppf(teoric_p))
log_norm_a = (pd.Series(np.exp(serie_z_norm*beta_log_norm + alfa_log_norm)))
#print(log_norm_a)
  # Cálculo de Datos Ajustados a Función Gumbel
gumbel_a = beta_gumbel - (1/alfa_gumbel) * np.log(np.log(1/teoric_p))
#print(gumbel_a)

Caudales_ajustados['Ajus Norm'] = norm_a
Caudales_ajustados['Ajus LogNor'] = log_norm_a
Caudales_ajustados['Ajus Gumbel'] = gumbel_a

print(Caudales_ajustados)